## 스태킹 알고리즘/스태킹 앙상블

- 단순 회귀나 랜덤포레스트를 이용할 경우 기존의 데이터에서 벗어난 범위의 예측을 진행할 경우 **외삽**문제가 발생함.


- 이럴때 사용할 수 있는게 스태킹 알고리즘이나 딥러닝 모델을 사용함.


- 스태킹 알고리즘 단계
***
1. n개의 모델로 학습 모델을 생성(기존의 회귀, rbf 등등...) 

2. n개의 모델에서 생성한 학습 모델을 기반으로 다시 최종 예측값(meta model) 생성
***

- 배깅과 비슷한 과정을 거치지만 최종 예측값을 생성한다는 것에서 차이점이 있음.

- 이렇게 적용한다고 하여 반드시 성능 향상을 이끌어낸다는 보장은 없음.

In [26]:
# 기본적인 스태킹 예제
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer_data = load_breast_cancer()

X_data = cancer_data.data
y_label = cancer_data.target

# 데이터 분리
X_training , X_testing , y_training , y_testing = train_test_split(X_data , y_label , test_size=0.2 , random_state=0)

In [27]:
print(X_training.shape)
print(X_testing.shape)
print(y_training.shape)
print(y_testing.shape)

(455, 30)
(114, 30)
(455,)
(114,)


In [28]:
# 개별 ML 모델을 위한 Classifier 생성.
knn_clf  = KNeighborsClassifier(n_neighbors=4) #K최근접이웃
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)#랜덤포레스트
dt_clf = DecisionTreeClassifier() #결정트리
ada_clf = AdaBoostClassifier(n_estimators=100) #아다부스트


In [29]:
# 개별 모델들을 학습. 
knn_clf.fit(X_training, y_training)  
rf_clf.fit(X_training , y_training)  
dt_clf.fit(X_training , y_training)
ada_clf.fit(X_training, y_training)

AdaBoostClassifier(n_estimators=100)

In [30]:
# 학습된 개별 모델들이 각자 반환하는 예측 데이터 셋을 생성하고 개별 모델의 정확도 측정. 
knn_pred = knn_clf.predict(X_testing)
rf_pred = rf_clf.predict(X_testing)
dt_pred = dt_clf.predict(X_testing)
ada_pred = ada_clf.predict(X_testing)

In [31]:
print('KNN 정확도: {0:.4f}'.format(accuracy_score(y_testing, knn_pred)))
print('랜덤 포레스트 정확도: {0:.4f}'.format(accuracy_score(y_testing, rf_pred)))
print('결정 트리 정확도: {0:.4f}'.format(accuracy_score(y_testing, dt_pred)))
print('에이다부스트 정확도: {0:.4f} :'.format(accuracy_score(y_testing, ada_pred)))

KNN 정확도: 0.9211
랜덤 포레스트 정확도: 0.9649
결정 트리 정확도: 0.9123
에이다부스트 정확도: 0.9561 :


4가지 모델중 랜덤 포레스트의 정확도가 가장 높게 나왔다.

스태킹 알고리즘은 4가지 모델의 결과값을 최종 모델의 새로운 인풋으로 전환함

In [32]:
# 4가지 모델 결과값
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])

In [33]:
# 예측 값을 4개의 변수로 전환
pred = np.transpose(pred)

In [34]:
pred.shape

(114, 4)

In [35]:
# 최종 Stacking 모델을 위한 Classifier생성. 
lr_final = LogisticRegression(C=10)

In [36]:
# 최종 모델 정확도
lr_final.fit(pred, y_testing)
final = lr_final.predict(pred)
print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(y_testing , final)))

최종 메타 모델의 예측 정확도: 0.9649


개별 모델과 같은 결과가 나왔다.

스태킹은 학습 데이터에 한하여 2번의 예측을 진행하므로 성능면에서 좋은 결과를 얻을 수 있으나, 학습 데이터에 분산이 큰 데이터가 있을 경우 성능이 떨어질 수가 있으며, 과적합에 대한 위험도를 배제할 수가 없다.

이럴때 CV를 포함한 스태킹 알고리즘을 실시한다.